In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [4]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312      1.0  76.722160                  NaN        NaN
0.000312      NaN        NaN                  1.0  38.907461
0.000750      2.0  40.961180                  2.0  33.904101
0.001052      3.0  33.041424                  3.0  31.686758
0.001476      4.0  26.177150                  4.0  28.647423
0.002070      5.0  20.453587                  5.0  24.715282
0.002904      6.0  15.887011                  6.0  20.074462
0.004074      7.0  12.355441                  7.0  15.169055
0.005714      8.0   9.670387                  8.0  10.650087
0.008015      9.0   7.632119                  9.0   7.176137
0.011243     10.0   6.059189                 10.0   5.028450
0.015771     11.0   4.800884                 11.0   3.956822
0.022122     12.0   3.733239                 12.0   3.369093
0.031031     13.0   2.741506                 13.0   2.795533
0.043528     14.0   1.535103                 14.0   1.821282
0.061057     15.0   0.621270                 15.0   0.940390
0.085645     16.0   1.009105                 16.0   1.169481
0.120136     17.0   1.978014                 17.0   1.896596
0.168516     18.0   2.672370                 18.0   2.339047
0.236378     19.0   3.351253                 19.0   2.848558
0.331549     20.0   4.134225                 20.0   3.729692
0.465100     21.0   5.128997                 21.0   5.076069
0.652400     22.0   6.668315                 22.0   6.960109
0.915100     23.0   8.412267                 23.0   8.662669
1.283650     24.0   8.111907                 24.0   7.946412
1.800600     25.0   6.297593                 25.0   5.854539
2.525700     26.0   4.914452                 26.0   4.720247
3.542800     27.0   3.934145                 27.0   4.198816
4.969550     28.0   3.201959                 28.0   3.476525
6.970850     29.0   2.668698                 29.0   2.688116
9.778100     30.0   2.244096                 30.0   2.196476
13.715850    31.0   1.822880                 31.0   2.033654
19.239350    32.0   1.385437                 32.0   1.708237
26.987250    33.0   1.051152                 33.0   1.211891
37.855300    34.0   0.824001                 34.0   0.818515
53.100050    35.0   0.576221                 35.0   0.619928
73.887500    36.0   0.318069                 36.0   0.429550
97.662500    37.0   0.177026                 37.0   0.244472
121.437500   38.0   0.131797                 38.0   0.136380
145.212500   39.0   0.032605                 39.0  -0.004834
168.987500   40.0  -0.100428                 40.0  -0.158854
192.762500   41.0  -0.096020                 41.0  -0.145611
216.537500   42.0   0.023673                 42.0  -0.003050
240.312500   43.0   0.075846                 43.0   0.056484
264.087500   44.0   0.091997                 44.0   0.071945
287.862500   45.0   0.098112                 45.0   0.078448
311.637500   46.0   0.099724                 46.0   0.082954
335.412500   47.0   0.098896                 47.0   0.086723
359.187500   48.0   0.096725                 48.0   0.088864
382.962500   49.0   0.093992                 49.0   0.089871
406.737500   50.0   0.091034                 50.0   0.089691
430.512500   51.0   0.087524                 51.0   0.088000
454.287500   52.0   0.082854                 52.0   0.083424
478.062500   53.0   0.076838                 53.0   0.076880
501.837500   54.0   0.070158                 54.0   0.068793
525.612500   55.0   0.063739                 55.0   0.060858
549.387500   56.0   0.058049                 56.0   0.053342
573.162500   57.0   0.053072                 57.0   0.046431
596.937500   58.0   0.048726                 58.0   0.040323
620.712500   59.0   0.044924                 59.0   0.035001
644.487500   60.0   0.041549                 60.0   0.030730
668.262500   61.0   0.038595                 61.0   0.027112
692.037500   62.0   0.036114                 62.0

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -17.779313   0.000000 -17.779313        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -17.458206   0.000182   
6.244000e-04 2     -17.779655   0.006016 -17.773639 -17.458570   0.003424   
8.759000e-04 3     -17.779811   0.007392 -17.772418 -17.458718   0.004583   
1.228600e-03 4     -17.780008   0.008970 -17.771038 -17.458913   0.006101   
1.723400e-03 5     -17.780256   0.010752 -17.769504 -17.459166   0.008034   
2.417400e-03 6     -17.780570   0.012747 -17.767822 -17.459497   0.010396   
3.390900e-03 7     -17.780969   0.014979 -17.765990 -17.459929   0.013144   
4.756500e-03 8     -17.781482   0.017490 -17.763991 -17.460498   0.016167   
6.672000e-03 9     -17.782148   0.020351 -17.761797 -17.461251   0.019337   
9.358900e-03 10    -17.783026   0.023658 -17.759368 -17.462257   0.022627   
1.312780e-02 11    -17.784201   0.027538 -17.756663 -17.463616   0.026231   
1.841450e-02 12    -17.785805   0.032149 -17.753656 -17.465488   0.030582   
2.583020e-02 13    -17.788054   0.037677 -17.750377 -17.468120   0.036173   
3.623230e-02 14    -17.791335   0.044337 -17.746999 -17.471927   0.043425   
5.082340e-02 15    -17.796754   0.052409 -17.744346 -17.478016   0.052663   
7.129060e-02 16    -17.806083   0.063244 -17.742839 -17.488079   0.065005   
1.000000e-01 17    -17.819859   0.080451 -17.739407 -17.502593   0.083497   
1.402710e-01 18    -17.837769   0.107797 -17.729972 -17.521261   0.111214   
1.967600e-01 19    -17.860581   0.148492 -17.712089 -17.544768   0.150374   
2.759970e-01 20    -17.888973   0.208339 -17.680634 -17.573624   0.205970   
3.871000e-01 21    -17.922659   0.296436 -17.626224 -17.607210   0.288647   
5.431000e-01 22    -17.958270   0.426827 -17.531443 -17.641600   0.416849   
7.617000e-01 23    -17.978681   0.619912 -17.358769 -17.659470   0.614968   
1.068500e+00 24    -17.941817   0.888772 -17.053045 -17.621985   0.892340   
1.498800e+00 25    -17.834572   1.195008 -16.639564 -17.517999   1.193441   
2.102400e+00 26    -17.688006   1.498725 -16.189282 -17.375251   1.469340   
2.949000e+00 27    -17.512420   1.815988 -15.696432 -17.202981   1.770493   
4.136600e+00 28    -17.309235   2.166257 -15.142978 -17.001061   2.159322   
5.802500e+00 29    -17.080558   2.569447 -14.511111 -16.771807   2.616188   
8.139200e+00 30    -16.822858   3.050440 -13.772418 -16.507656   3.096182   
1.141700e+01 31    -16.527399   3.626315 -12.901084 -16.198303   3.639762   
1.601470e+01 32    -16.200522   4.292234 -11.908288 -15.849884   4.399046   
2.246400e+01 33    -15.884825   5.034963 -10.849862 -15.510466   5.364798   
3.151050e+01 34    -15.603034   5.879609  -9.723424 -15.200379   6.353535   
4.420010e+01 35    -15.321586   6.836779  -8.484808 -14.881191   7.264844   
6.200000e+01 36    -15.069403   7.799571  -7.269832 -14.583169   8.274090   
8.577500e+01 37    -14.997598   8.623551  -6.374047 -14.470869   9.371664   
1.095500e+02 38    -15.099057   9.223570  -5.875487 -14.536593  10.125970   
1.333250e+02 39    -15.263683   9.759378  -5.504305 -14.660642  10.634148   
1.571000e+02 40    -15.604738  10.192260  -5.412479 -14.973981  10.933872   
1.808750e+02 41    -16.352852  10.657536  -5.695316 -15.739163  11.251624   
2.046500e+02 42    -17.440965  11.475226  -5.965739 -16.878482  11.980812   
2.284250e+02 43    -18.588967  12.689898  -5.899069 -18.079507  13.173245   
2.522000e+02 44    -19.724622  14.039158  -5.685464 -19.260894  14.513725   
2.759750e+02 45    -20.842292  15.415922  -5.426370 -20.418854  15.874325   
2.997500e+02 46    -21.940873  16.790817  -5.150056 -21.553694  17.230121   
3.235250e+02 47    -23.019646  18.150445  -4.869201 -22.663461  18.573538   
3.473000e+02 48    -24.077740  19.487061  -4.590679 -23.749667  19.904010   
3.710750e+02 49    -25

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')